# Notebook for report figures construction

This notebook is meant to be run on Google colab, otherwise paths to subject data and modules might need adjustments

In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smplx
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx
    !git clone https://github.com/vchoutas/smplx
    %cd smplx
    !python setup.py install
    %cd ..


# detectron2
need_detectron=False
try:
    import detectron2
except ModuleNotFoundError:
    need_detectron=True   

if need_detectron:
    !git clone https://github.com/facebookresearch/detectron2.git detectron2_repo
    !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

In [ ]:
### Download data if not done yet

import os
import zipfile
import urllib.request as request

attributes = ['body', 'body_texture']

subject = 1

if not os.path.exists('subject_%d' % subject):
    for attr in attributes:
        url = os.path.join('https://humbi-dataset.s3.amazonaws.com', attr + '_subject', 'subject_%d.zip' % subject)
        path = '%s_subject_%d.zip' % (attr, subject)
        request.urlretrieve(url, path)
        downloaded_zip = zipfile.ZipFile(path)
        downloaded_zip.extractall() # !unzip downloaded_zip
        os.remove(path)

In [ ]:
import smplx
import numpy as np
import matplotlib.pyplot as plt
from torchvision.io import read_image

from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes, packed_to_list
from pytorch3d.transforms import axis_angle_to_matrix
from pytorch3d.renderer import PerspectiveCameras, TexturesUV, TexturesVertex

from utils.plot_structures import plot_structure
from utils.smpl_to_smplx import extract_smpl_param, humbi_smpl_mesh
from utils.camera_calibration import get_camera_parameters
from utils.renderers import get_renderers
from utils.pointrend_segmentation import get_pointrend_segmentation

In [ ]:
### Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
### Global arguments
pose = '00000025' # T-pose

smpl_obj = drive_path + 'smplx/smpl_uv.obj'
smplx_obj = drive_path + 'smplx/smplx_uv.obj'

save_path = drive_path + 'tmp_files/figures'

smplx_model_path = drive_path + 'smplx'
smplx_model = smplx.SMPLXLayer(smplx_model_path, gender='neutral').to(device)

In [ ]:
### Construct TexturesUV object
def construct_textures_uv(filepath:str, obj_path:str):
    texture = read_image(filepath)
    texture = torch.moveaxis(texture, 0, 2).unsqueeze(0).to(device).float() * 1.0/255

    default_mesh = load_obj(obj_path, load_textures=False)
    verts_uvs = default_mesh[2].verts_uvs.to(device).unsqueeze(0).to(device)
    faces_uvs = default_mesh[1].textures_idx.to(device).unsqueeze(0).to(device)

    texture_uv = TexturesUV(maps=texture, faces_uvs=faces_uvs, verts_uvs=verts_uvs)

    return texture_uv

In [ ]:
### Convert TexturesUV to TexturesVertex
def convert_to_textureVertex(textures_uv: TexturesUV, meshes:Meshes) -> TexturesVertex:
    verts_colors_packed = torch.zeros_like(meshes.verts_packed())
    verts_colors_packed[meshes.faces_packed()] = textures_uv.faces_verts_textures_packed().to(device)
    return TexturesVertex( packed_to_list(verts_colors_packed, meshes.num_verts_per_mesh()) )

In [ ]:
def renders(subject, pose, mesh, camera_idx):
    # Extract camera parameters
    R, T, f, p = get_camera_parameters(subject, camera_idx)

    # Construct camera
    cameras = PerspectiveCameras(focal_length=-f, principal_point=p, R=R, T=T, in_ndc=False, image_size=((1080, 1920),), device=device)
    
    # Render mesh from camera viewpoint
    silhouette_renderer, phong_renderer = get_renderers(cameras, (1080, 1920), device=device)
    phong_render = phong_renderer(mesh)
    silhouette_render = silhouette_renderer(mesh)

    rgb_render = phong_render[0, ..., :3]
    silh_render = silhouette_render[0, ..., 3]

    return silh_render, rgb_render

# SMPL-X model construction

In [ ]:
### Extract geometry parameters
geometry_filename = drive_path + 'humbi_output/humbi_smplx_npz/output_subject_%d.npz' % subject
npzfile = np.load(geometry_filename)

# stored into npz file
body_pose = torch.Tensor(npzfile['body_pose']).to(device)
betas = torch.Tensor(npzfile['betas']).to(device)
scale = torch.Tensor(npzfile['scale']).to(device)
verts_disps = torch.Tensor(npzfile['verts_disps']).to(device)
learned_geometry = torch.Tensor(npzfile['learned_geometry']).to(device)

# extract from smpl
transl, global_orient = extract_smpl_param(subject, pose)[1:3]

In [ ]:
### Construct SMPL-X model
smplx_faces = torch.Tensor(smplx_model.faces.astype('int')).type(torch.int32).unsqueeze(0).to(device)

smplx_verts = smplx_model.forward(global_orient=axis_angle_to_matrix(global_orient),
                                  body_pose=axis_angle_to_matrix(body_pose),
                                  betas=betas)['vertices'].to(device)

smplx_mesh = Meshes(smplx_verts * scale + transl, smplx_faces)

smplx_txt_path = drive_path + 'humbi_output/humbi_smplx_rgb/rgb_texture_%d.png' % subject
smplx_texture = construct_textures_uv(smplx_txt_path, smplx_obj)

verts_smplx_disp = (smplx_verts * scale) + (smplx_mesh.verts_normals_packed() * verts_disps).unsqueeze(0)
smplx_mesh = Meshes(verts_smplx_disp + transl, smplx_faces, smplx_texture)

In [ ]:
### Default SMPL-X model
init_verts = smplx_model.forward()['vertices'].to(device)
init_mesh = Meshes(init_verts, smplx_faces)

In [ ]:
### SMPL-X mesh with TexturesVertex for interactive plotting
smplx_texture_verts = convert_to_textureVertex(smplx_texture, smplx_mesh)
smplx_mesh_plot = Meshes(smplx_mesh.verts_padded(), smplx_mesh.faces_padded(), smplx_texture_verts)

# SMPL model construction

In [ ]:
smpl_mesh = humbi_smpl_mesh(subject, pose)

smpl_txt_path = 'subject_%d/body/%s/appearance/median_map.png' % (subject, pose)
smpl_texture = construct_textures_uv(smpl_txt_path, smpl_obj)

smpl_mesh = Meshes(smpl_mesh.verts_padded(), smpl_mesh.faces_padded(), smpl_texture)

In [ ]:
### SMPL mesh with TexturesVertex for interactive plotting
smpl_texture_verts = convert_to_textureVertex(smpl_texture, smpl_mesh)
smpl_mesh_plot = Meshes(smpl_mesh.verts_padded(), smpl_mesh.faces_padded(), smpl_texture_verts)

# Plotting and Saving

In [ ]:
### Interactive plotting
plot_structure([smpl_mesh_plot, smplx_mesh_plot])

In [ ]:
### Store SMPL and initial SMPL-X mesh as .obj files
os.makedirs(save_path, exist_ok=True)
smpl_filename = os.path.join(save_path, 'smpl_mesh.obj')
smplx_filename = os.path.join(save_path, 'smplx_mesh.obj')

save_obj(smpl_filename, verts=smpl_mesh.verts_packed(), faces=smpl_mesh.faces_packed())
save_obj(smplx_filename, verts=init_mesh.verts_packed(), faces=init_mesh.faces_packed())

In [ ]:
### Segment photoscans and render scene
camera_idx = 2 #, 30, 62

# Segment person in photo from camera viewpoint
photo_path = 'subject_%s/body/%s/image/image%s.jpg' % (subject, pose, str(camera_idx).zfill(7))
photo, silh_photo, rgb_photo = get_pointrend_segmentation(photo_path, device=device)

smpl_silh, smpl_rgb = renders(subject, pose, smpl_mesh, camera_idx)
smplx_silh, smplx_rgb = renders(subject, pose, smplx_mesh, camera_idx)

# Storing paths for images
os.makedirs(save_path, exist_ok=True)
photo_path = os.path.join(save_path, 'photo_%s.png' %str(camera_idx).zfill(7))
rgb_photo_path = os.path.join(save_path, 'segm_photo_%s.png' %str(camera_idx).zfill(7))
silh_photo_path = os.path.join(save_path, 'segm_mask_%s.png' %str(camera_idx).zfill(7))
smpl_rgb_path = os.path.join(save_path, 'smpl_render_%s.png' %str(camera_idx).zfill(7))
smplx_rgb_path = os.path.join(save_path, 'smplx_render_%s.png' %str(camera_idx).zfill(7))
smpl_silh_path = os.path.join(save_path, 'smpl_mask_%s.png' %str(camera_idx).zfill(7))
smplx_silh_path = os.path.join(save_path, 'smplx_mask_%s.png' %str(camera_idx).zfill(7))

In [ ]:
# Target Image
plt.figure(figsize=(16, 9))
plt.imshow(photo.cpu())
plt.imsave(photo_path, photo.cpu().numpy())
plt.axis("off")

# RGB Photo
plt.figure(figsize=(16, 9))
plt.imshow(rgb_photo[0].cpu())
plt.imsave(rgb_photo_path, rgb_photo[0].cpu().numpy())
plt.axis("off")

# RGB SMPL Render
plt.figure(figsize=(16, 9))
plt.imshow(smpl_rgb.cpu())
plt.imsave(smpl_rgb_path, smpl_rgb.cpu().numpy())
plt.axis("off")

# RGB SMPL-X Render
plt.figure(figsize=(16, 9))
plt.imshow(smplx_rgb.cpu())
plt.imsave(smplx_rgb_path, smplx_rgb.cpu().numpy())
plt.axis("off")

# Mask Photo
plt.figure(figsize=(16, 9))
plt.imshow(silh_photo[0].cpu())
plt.imsave(silh_photo_path, silh_photo[0].cpu().numpy())
plt.axis("off")

# Mask SMPL Render
plt.figure(figsize=(16, 9))
plt.imshow(smpl_silh.cpu())
plt.imsave(smpl_silh_path, smpl_silh.cpu().numpy())
plt.axis("off")

# Mask SMPL-X Render
plt.figure(figsize=(16, 9))
plt.imshow(smplx_silh.cpu())
plt.imsave(smplx_silh_path, smplx_silh.cpu().numpy())
plt.axis("off")